# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [202]:
import pandas as pd
import numpy as np

def valorparabooleano(x):
    if type(x) is str:
        if x in ('2','3','4','5'):
            return True
        else:
            return False
    else:
        return False




In [170]:
#1 Seu código aqui:

propostas = pd.read_csv('application_record.csv')
pg = pd.read_csv('pagamentos_largo.csv')

In [214]:
pg_booleano = pg.iloc[:,1:].map(valorparabooleano)
print(pg_booleano)

       mes_0  mes_1  mes_10  mes_11  mes_12  mes_13  mes_14  mes_15  mes_16  \
0      False  False   False   False   False   False   False   False   False   
1      False  False   False   False   False   False   False   False   False   
2      False  False   False   False   False   False   False   False   False   
3      False  False   False   False   False   False   False   False   False   
4      False  False   False   False   False   False   False   False   False   
...      ...    ...     ...     ...     ...     ...     ...     ...     ...   
20932  False  False   False   False   False   False   False   False   False   
20933  False  False   False   False   False   False   False   False   False   
20934  False  False   False   False   False   False   False   False   False   
20935  False  False   False   False   False   False   False   False   False   
20936  False  False   False   False   False   False   False   False   False   

       mes_17  ...  mes_22  mes_23  mes_24  mes_3  

Este dataframe resolve nosso primeiro exercício, associando, para cada mês, um True, caso o cliente esteja em default naquele mês (acúmulo de mais de 60 dias de atraso, ou seja, de 2 a 5 na nossa legenda) e False caso contrário...

In [260]:
# 2) Seu código aqui

booleano_default24m = pg_booleano.iloc[:,1:].sum(axis=1)
booleano_default24m = booleano_default24m > 0
pg[booleano_default24m]

# Os clientes mostrados abaixo, são apenas aqueles que apresentaram comportamento de default em ao menos um dos 24 meses (ou seja, cuja soma do nosso
# pg booleano na linha correspondente a eles recebeu resultado maior que zero, o que significa que havia ao menos um True entre as situações de default)

0        False
1        False
2        False
3        False
4        False
         ...  
20932    False
20933    False
20934    False
20935    False
20936    False
Length: 20937, dtype: bool


,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
130,5002126,0,0,C,C,C,C,C,C,C,...,C,C,C,NaN,1,2,2,3,3,2
177,5002283,0,0,1,2,0,0,0,0,0,...,C,C,C,0,0,0,0,0,0,0
303,5002642,0,0,0,0,1,0,0,0,C,...,C,C,C,0,0,0,0,0,1,2
355,5002795,0,1,0,0,C,C,C,C,C,...,C,C,C,0,0,0,1,2,0,0
556,5003267,1,2,C,C,C,C,C,C,C,...,C,C,C,3,1,1,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20523,5149192,0,0,5,5,5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5,5,5,5,5,5,5
20734,5149698,0,0,NaN,NaN,NaN,4,2,2,0,...,NaN,NaN,NaN,0,NaN,0,1,NaN,2,0
20744,5149728,0,0,0,0,2,C,C,C,C,...,C,NaN,NaN,0,0,0,0,0,0,0
20745,5149730,0,0,0,0,2,C,C,C,C,...,C,NaN,NaN,0,0,0,0,0,0,0


In [256]:
# 3) Seu código aqui
# Para cumprir nosso propósito, notemos que: temos duas bases de dados, uma mais extensa, e outra menor. A base menor (a de pagamentos) possui apenas
# pessoas que compraram o cartão de crédito. Queremos filtrar da base maior aqueles clientes que não possuem cartão de crédito. Isto é, não estão na base
# de pagamentos, visto que para estes clientes não faz sentido analisar o default... 

df_proponentes = pd.read_csv('application_record.csv')
print(df_proponentes.shape)
colunas = df_proponentes.columns
df_expdef = df_proponentes.merge(pg, on='ID', how='inner')
# Como fizemos uma junção, as colunas de meses vieram junto. Para excluí-las, redefiniremos nosso dataframe tomando apenas as colunas que já estavam no
# original.
df_expdef = df_expdef[colunas]
print(df_expdef)

print(df_expdef.shape)

# Podemos ver no shape do df_expdef (df expostos ao default) que agora nosso dataframe possui a quantidade certa de colunas. Ou seja, a mesma que
# inicialmente, porém agora apenas com os expostos ao default.

(438557, 18)
            ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0      5008806           M            Y               Y             0   
1      5008810           F            N               Y             0   
2      5008811           F            N               Y             0   
3      5112956           M            Y               Y             0   
4      5008825           F            Y               N             0   
...        ...         ...          ...             ...           ...   
16645  5149042           F            N               Y             0   
16646  5149056           F            N               Y             0   
16647  5149145           M            Y               Y             0   
16648  5149158           M            Y               Y             0   
16649  5149838           F            N               Y             0   

       AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0              112500.0       

In [266]:
# 4) eu código aqui
# Lá em cima, já haviamos contruído uma variável que armazenava se o cliente havia entrado em default nos últimos 24m. Esta variável era:
# booleano_default24m. Agora, podemos apenas criar uma nova coluna no nosso df_expdef que armazena esse valor. Como é de costume nos dataframes que já
# vimos anteriormente, armazenaremos nosso default na forma de 0 para False e 1 para True, fazendo simplesmente booleano_default24m.astype(int)

df_expdef['default'] = booleano_default24m.astype(int)

print(df_expdef)

# Deste modo, encerramos deixando nosso dataframe com a "cara" que era normalmente vista nas bases de dados nas quais trabalhamos anteriormente!

            ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0      5008806           M            Y               Y             0   
1      5008810           F            N               Y             0   
2      5008811           F            N               Y             0   
3      5112956           M            Y               Y             0   
4      5008825           F            Y               N             0   
...        ...         ...          ...             ...           ...   
16645  5149042           F            N               Y             0   
16646  5149056           F            N               Y             0   
16647  5149145           M            Y               Y             0   
16648  5149158           M            Y               Y             0   
16649  5149838           F            N               Y             0   

       AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0              112500.0               Worki

In [276]:
# Tarefa 5
# Esta tarefa, se não me engano, eu já havia obtido o resultado dela acima, a menos que esteja entendendo errado... se o que o exercício pede (como eu 
# estou interpretando) a contagem de clientes em default, podemos simplesmente fazer:

sum(booleano_default24m)
# Ou seja, temos 429 clientes em default... O que é razoável, visto que teríamos:

percentual_default = 429/16650
print(percentual_default)

# ou seja, 0,026% de clientes em default.

0.025765765765765766
